In [1]:
import altair as alt
import pandas as pd

In [2]:
df = pd.read_csv('AmazonBooksSheet1.csv')
df.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction


In [3]:
df_1 = df.copy(deep=False)
df_1.drop(["Name", "Author", "User Rating", "Price", "Reviews"], axis=1, inplace=True)
df_1.head()

,Year,Genre
0,2016,Non Fiction
1,2011,Fiction
2,2018,Non Fiction
3,2017,Fiction
4,2019,Non Fiction


In [4]:
df_1.columns = ["Year", 'Genre']
df_1pivoted = df_1.pivot_table(columns=['Year','Genre'], aggfunc='size')
df_1pivoted = pd.DataFrame(df_1pivoted).reset_index()

df_1pivoted.columns = ["Year", "Genre", "Val"]
df_1pivoted.head()

,Year,Genre,Val
0,2009,Fiction,24
1,2009,Non Fiction,26
2,2010,Fiction,20
3,2010,Non Fiction,30
4,2011,Fiction,21


In [5]:
select_year = alt.selection_single(on = 'click', fields = ['Year'], nearest = False, empty = 'all')

In [6]:
final_chart1 = alt.Chart(df_1pivoted).mark_bar().encode(
    x = alt.X('Year:N', title="Year"),
    y = alt.Y('Val:Q', title="Amount"), 
    color = alt.Color('Genre:N', title="Genre" ),
    tooltip = [
        alt.Tooltip('Val:Q', title="Amount"),
        alt.Tooltip('Genre:N'),
    ],
    order=alt.Order('Val:Q',
                    sort='descending'),
    opacity = alt.condition(
        select_year,
        alt.value(1),
        alt.value(0.5)
    )
).add_selection(
    select_year
).properties(width = 650, 
              height = 350, 
              background = '#F9F9F9', 
              padding = 25,
              title={
              "text": ["Correlation between the amount of books and genre per year"], 
              "subtitle": ["Click on a bar to highlight it"],
              "color": "black",
              "subtitleColor": "darkgrey"
            }
             ).configure_axis(
                              gridWidth = 1,
                              titleFontSize = 12,
                              labelFontSize = 12
             ).configure_title(
                                fontSize = 14,
                                fontWeight = 'bold'
             ).configure_legend(
                                titleFontSize=14,
                                labelFontSize=12
             ).configure_axisX(labelPadding=5, 
                  labelAngle = 0,
                              )
final_chart1

alt.Chart(...)

Цим графіком я хотіла порівняти частки жанрів у загальній кількості топ-50 книжок кожного року. З графіка видно, що у топ книжок частіше потрапляють книги жанру фікшн ніж нон-фікшин, і лише у 2014 році сталося навпаки. 


Для виконання цього завдання, я також думала використовувати піраміду з барів, проте той графік був доволі сладний для розуміння. З нього було важче з першого погляду зрозуміти, який жанр переважає. 

Щодо цього графіку, то він простіший навигляд, не навантажений даними, одразу видно. який жанр переважає та наскільки. Також завдяки інтерактивності можна виділити один з барів і легко його порівняти з іншими.

In [7]:
df_2 = df.copy(deep=False)
df_2.drop(["Name", "Author", "Genre"], axis=1, inplace=True)
df_2.head()

,User Rating,Reviews,Price,Year
0,4.7,17350,8,2016
1,4.6,2052,22,2011
2,4.7,18979,15,2018
3,4.7,21424,6,2017
4,4.8,7665,12,2019


In [8]:
df_2['User Rating'].nunique()

14

In [10]:
years = [str(i) for i in df_2["Year"].unique()]
years.sort()
df_2["Year"] = df_2["Year"].astype(str)

input_dropdown = alt.binding_select(options = years)
selectYear = alt.selection_single(fields = ['Year'], bind = input_dropdown, name = 'Select', empty = 'all')

alt.Chart(df_2).mark_circle().add_selection(
    selectYear
).encode(
    x = alt.X('User Rating:Q', 
              scale = alt.Scale(domain = [df_2["User Rating"].min(), 
                                          df_2["User Rating"].max()])),
    y = alt.Y('Price:Q', 
              scale = alt.Scale(zero = False,
                                domain = [df_2["Price"].min(), 
                                          df_2["Price"].max()])),
    color=alt.condition(
    selectYear,
    alt.value("blue"),
    alt.value("grey")
    ),
    size = alt.Size('Reviews:Q', 
                    scale = alt.Scale(domain = [df_2.Reviews.min(), 
                                               df_2.Reviews.max()])),
    tooltip = [
        alt.Tooltip('User Rating:Q'),
        alt.Tooltip('Price:Q'),
    ],
    opacity=alt.condition(selectYear, alt.value(0.75), alt.value(0.15))
).properties(width = 650, 
              height = 350, 
              background = '#F9F9F9', 
              padding = 25,
              title={
              "text": ["Prices of books with different ratings by years"], 
              "subtitle": ["Does the best book cost the most?", "Select a year on the selector to highlight it"],
              "color": "black",
              "subtitleColor": "darkgrey"
              }
             ).configure_axis(
                              gridWidth = 1,
                              titleFontSize = 12,
                              labelFontSize = 12
             ).configure_title(
                                fontSize = 14,
                                fontWeight = 'bold'
             ).configure_legend(
                                titleFontSize=14,
                                labelFontSize=12
             ).configure_axisX(labelPadding=5, 
                  labelAngle = 0,
                              )

alt.Chart(...)

Цим графіком я хотіла дослідити чи є залежність між ціною книги та її оцінкою, тоюто чи справді найкраща книга є найдорожчою, чи можливо хороші книги трапляються серед дешевших. 

Для цього завдання я розглядала також варіанти представлення як лінійний графік, проте, мені здається, що у цьому графіку з точками, легше порівняти ситуацію за роками. Завдяки селектору, можна вибрати один рік, і значення цього року виділяться синім, тоді як інші дані стануть сірими, це дає змогу порівняти роки. Також точки допомагають помітити скупчення даних (книг в одній ціновій категорії, книг зі схожими оцінками). 

Проте мінусами цієї подачі є те, що графік виглядає доволі брудно і навантажено.

In [11]:
df_3 = df.copy(deep=False)
df_3.drop(["Name", "Author", "Price", "Reviews"], axis=1, inplace=True)
df_3.head()

,User Rating,Year,Genre
0,4.7,2016,Non Fiction
1,4.6,2011,Fiction
2,4.7,2018,Non Fiction
3,4.7,2017,Fiction
4,4.8,2019,Non Fiction


In [12]:
df_3_avprice = df_3.pivot_table(columns=['Year','Genre'])

df_3_avprice = pd.DataFrame(df_3_avprice).reset_index()
df_3_avprice = df_3_avprice.melt()
df_3_avprice = df_3_avprice.drop([0], axis=0)

df_3_avprice["value"] = df_3_avprice["value"].apply(lambda x: round(x, 2))

df_3_avprice.head()

,Year,Genre,value
1,2009,Fiction,4.59
2,2009,Non Fiction,4.58
3,2010,Fiction,4.62
4,2010,Non Fiction,4.52
5,2011,Fiction,4.62


In [13]:
line_chart = alt.Chart().mark_line(point=True).encode(
    x = alt.X('Year:N'), 
    y = alt.Y('value:Q', title="Users Rating", scale = alt.Scale(
                  zero = False
              )),
    color = alt.Color('Genre:N'),
)

label = alt.selection_single(encodings=['x'], on='mouseover', nearest=True, empty='none')


final_chart = alt.layer(line_chart, 
    
    alt.Chart().mark_rule(color='#aaa').encode(
        x='Year:N'
    ).transform_filter(label),
    
    line_chart.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    line_chart.mark_text(align='left', dx=5, dy=-8, stroke='white', strokeWidth=2).encode(
        text='value:Q'
    ).transform_filter(label),

    line_chart.mark_text(align='left', dx=5, dy=-8).encode(
        text='value:Q'
    ).transform_filter(label),
    
    data=df_3_avprice
).properties(width = 650, 
              height = 350, 
              background = '#F9F9F9', 
              padding = 25,
              title={
              "text": ["Average rating for fictional and non fictional books"], 
              "subtitle": ["Mouse on a year to highlight it"],
              "color": "black",
              "subtitleColor": "darkgrey"
              }
             ).configure_axis(
                              gridWidth = 1,
                              titleFontSize = 12,
                              labelFontSize = 12
             ).configure_title(
                                fontSize = 14,
                                fontWeight = 'bold'
             ).configure_legend(
                                titleFontSize=14,
                                labelFontSize=12
             ).configure_axisX(labelPadding=5, 
                  labelAngle = 0,
                              )
final_chart

alt.LayerChart(...)

Тут я вирішила показати середні оцінки книг в залежності від жанру. З графіка можна зробити висновок, що люди переважно краще оцінюють книги жанру фікшн, і лише у 2012-2013 та 2020 роках кращими були нон-фікшн. 

Для виконання цієї задачі я пробувала використати теплову карту (heatmap), проте через те, що діапазон значень оцінок доволі вузький, кодування кольором є не найкращим варіантом. Тому я використала лінійний графік. Цей графік простий для зчитування та виглядає охайно і не навантажено. Також, якщо навести мишкою на точки, то можна побачити значення оцінок по жанрах певного року, це спрощує визначення точної оцінки по графіку.